# Crime Data from St. Louis Missouri (2008-2015)
## Loading and Cleaning the Data

In [1]:
import gmaps
import numpy as np
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [2]:
# Prepare list of files to load.
load_urls = ['2008_data.csv', '2009.csv', '2010.csv', '2011.csv', '2012.csv', '2013.csv', '2014.csv', '2015.csv']

# Create list of columns we want to use in the final DF.
column_names = ['CADAddress', 'CADStreet', 'CodedMonth', 'Count', 'Crime', 'ShortCrimeCode', 'UCRType',\
                'UCRCrime', 'DateOccured', 'Description', 'District', 'FlagCrime', 'FlagUnfounded',\
                'ILEADSAddress','ILEADSStreet', 'LocationComment', 'LocationName', 'Neighborhood',\
                'NeighborhoodName', 'NeighborhoodPrimaryDistrict', 'NeighborhoodAddlDistrict', 'Latitude',\
                'Longitude', 'Year']

In [4]:
# Create empty DF for crime data
crime_df = pd.DataFrame(columns = column_names)

# Read single year crime files and store into final data frame
for file in load_urls:
    load_df = pd.read_csv(f'Resources/{file}')
    
    # Select only the columns we want from the single year csv.
    load_df = load_df[column_names]
    
    # Append data from the single year csv into the final DF
    crime_df = crime_df.append(load_df,ignore_index=True)

In [5]:
# Preview data
crime_df.head()

,CADAddress,CADStreet,CodedMonth,Count,Crime,ShortCrimeCode,UCRType,UCRCrime,DateOccured,Description,...,ILEADSStreet,LocationComment,LocationName,Neighborhood,NeighborhoodName,NeighborhoodPrimaryDistrict,NeighborhoodAddlDistrict,Latitude,Longitude,Year
0,4900,MAPLE,2008-04,1,177000,17,2,Sex Offenses,1/1/2000,SEX OFFNS-INCEST,...,MAPLE AVE,NaN,NaN,53.0,Fountain Park,8.0,NaN,38.658845,-90.260116,2008.0
1,4357,CALIFORNIA,2008-04,1,67501,6,1,Larceny-theft,1/1/2003,LARCENY-FROM BUILDING OVER $25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4313,MARGARETTA,2008-04,1,115400,11,2,Fraud,1/1/2004,STLG BY DECEIT/IDENTITY THEFT REPORT,...,MARGARETTA AVE,NaN,NaN,68.0,O'Fallon,5.0,6.0,38.671754,-90.233544,2008.0
3,3939,OHIO,2008-04,1,115400,11,2,Fraud,1/1/2006,STLG BY DECEIT/IDENTITY THEFT REPORT,...,OHIO AVE,NaN,NaN,16.0,Dutchtown,1.0,NaN,38.583504,-90.230145,2008.0
4,2612,ANNIE MALONE DR,2008-04,1,91113,9,2,Other assaults,1/1/2007,SIMPLE ASSAULT-ADULT/NO INJURY,...,ANNIE MALONE DR,@ANNIE MALONE HOME,NaN,57.0,The Ville,8.0,NaN,38.659580,-90.236606,2008.0


In [6]:
# How many records do we have?
crime_df['CADAddress'].count()

449000

In [7]:
# Remove rows that do not have location information complete.
crime_df = crime_df.loc[(pd.isna(crime_df['Latitude'])==False) & (pd.isna(crime_df['Longitude'])==False) & (pd.isna(crime_df['NeighborhoodName'])==False)]

In [8]:
# How many records do we have after dropping nulls?
crime_df['CADAddress'].count()

435650

In [9]:
# I had issues with indexing into the list created by the split string funtion for reach row so I created a new column for the
## split string then did a for loop to assign the year from that column to the year column if the year column is null.

crime_df['Year_calc'] = crime_df['CodedMonth'].str.split('-')

for index in crime_df.index:
    if pd.isna(crime_df['Year'][index]) == True:
        crime_df['Year'][index] =  crime_df['Year_calc'][index][0]
crime_df.head()

,CADAddress,CADStreet,CodedMonth,Count,Crime,ShortCrimeCode,UCRType,UCRCrime,DateOccured,Description,...,LocationComment,LocationName,Neighborhood,NeighborhoodName,NeighborhoodPrimaryDistrict,NeighborhoodAddlDistrict,Latitude,Longitude,Year,Year_calc
0,4900,MAPLE,2008-04,1,177000,17,2,Sex Offenses,1/1/2000,SEX OFFNS-INCEST,...,NaN,NaN,53.0,Fountain Park,8.0,NaN,38.658845,-90.260116,2008.0,"[2008, 04]"
2,4313,MARGARETTA,2008-04,1,115400,11,2,Fraud,1/1/2004,STLG BY DECEIT/IDENTITY THEFT REPORT,...,NaN,NaN,68.0,O'Fallon,5.0,6.0,38.671754,-90.233544,2008.0,"[2008, 04]"
3,3939,OHIO,2008-04,1,115400,11,2,Fraud,1/1/2006,STLG BY DECEIT/IDENTITY THEFT REPORT,...,NaN,NaN,16.0,Dutchtown,1.0,NaN,38.583504,-90.230145,2008.0,"[2008, 04]"
4,2612,ANNIE MALONE DR,2008-04,1,91113,9,2,Other assaults,1/1/2007,SIMPLE ASSAULT-ADULT/NO INJURY,...,@ANNIE MALONE HOME,NaN,57.0,The Ville,8.0,NaN,38.659580,-90.236606,2008.0,"[2008, 04]"
5,5500,DELMAR,2008-04,1,21000,2,1,Forcible Rape,1/1/2008,RAPE -- FORCIBLE,...,NaN,NaN,49.0,Visitation Park,7.0,NaN,38.654057,-90.278155,2008.0,"[2008, 04]"


In [10]:
# Remove the no longer needed Year_calc column
crime_df.drop('Year_calc', inplace=True, axis=1)

In [11]:
# Change the data type of the year column to be an int to remove the decimal
crime_df['Year'] = crime_df['Year'].astype('int')

crime_df.head()

,CADAddress,CADStreet,CodedMonth,Count,Crime,ShortCrimeCode,UCRType,UCRCrime,DateOccured,Description,...,ILEADSStreet,LocationComment,LocationName,Neighborhood,NeighborhoodName,NeighborhoodPrimaryDistrict,NeighborhoodAddlDistrict,Latitude,Longitude,Year
0,4900,MAPLE,2008-04,1,177000,17,2,Sex Offenses,1/1/2000,SEX OFFNS-INCEST,...,MAPLE AVE,NaN,NaN,53.0,Fountain Park,8.0,NaN,38.658845,-90.260116,2008
2,4313,MARGARETTA,2008-04,1,115400,11,2,Fraud,1/1/2004,STLG BY DECEIT/IDENTITY THEFT REPORT,...,MARGARETTA AVE,NaN,NaN,68.0,O'Fallon,5.0,6.0,38.671754,-90.233544,2008
3,3939,OHIO,2008-04,1,115400,11,2,Fraud,1/1/2006,STLG BY DECEIT/IDENTITY THEFT REPORT,...,OHIO AVE,NaN,NaN,16.0,Dutchtown,1.0,NaN,38.583504,-90.230145,2008
4,2612,ANNIE MALONE DR,2008-04,1,91113,9,2,Other assaults,1/1/2007,SIMPLE ASSAULT-ADULT/NO INJURY,...,ANNIE MALONE DR,@ANNIE MALONE HOME,NaN,57.0,The Ville,8.0,NaN,38.659580,-90.236606,2008
5,5500,DELMAR,2008-04,1,21000,2,1,Forcible Rape,1/1/2008,RAPE -- FORCIBLE,...,DELMAR BLVD,NaN,NaN,49.0,Visitation Park,7.0,NaN,38.654057,-90.278155,2008


In [12]:
# Remove row where CADStreet is unknown
crime_df = crime_df.loc[crime_df['CADStreet'] != 'unknown 0000']

In [13]:
# Check if we can map the Nieghborhood column to a value from another column from the same street, if null.

neighborhoods = crime_df.groupby(['CADStreet','NeighborhoodName']).count()
neighborhoods

## Looks like CADStreet and Neighboborhood do not have a 1-1 relationship, as suspected. So we should not try to map this.
### When we do analysis on Neighborhoods we will just need to keep that in mind. 

CADAddress  CodedMonth  Count  Crime  \
CADStreet NeighborhoodName                                         
          Academy                  867         867    867    867   
          Baden                   1249        1249   1249   1249   
          Benton Park              607         607    607    607   
          Benton Park West         723         723    723    723   
          Bevo Mill               1160        1160   1160   1160   
...                                ...         ...    ...    ...   
ZEALAND   Midtown                    1           1      1      1   
          The Great Ville            1           1      1      1   
ZELLIE    Cardonlet                  3           3      3      3   
alero     The Hill                   1           1      1      1   
hilton    Downtown                   1           1      1      1   

                            ShortCrimeCode  UCRType  UCRCrime  DateOccured  \
CADStreet NeighborhoodName                                                   
          Academy                      867      867       867          867   
          Baden                       1249     1249      1249         1249   
          Benton Park                  607      607       607          607   
          Benton Park West             723      723       723          723   
          Bevo Mill                   1160     1160      1160         1160   
...                                    ...      ...       ...          ...   
ZEALAND   Midtown                        1        1         1            1   
          The Great Ville                1        1         1            1   
ZELLIE    Cardonlet                      3        3         3            3   
alero     The Hill                       1        1         1            1   
hilton    Downtown                       1        1         1            1   

                            Description  District  ...  ILEADSAddress  \
CADStreet NeighborhoodName                         ...                  
          Academy                   867       867  ...            867   
          Baden                    1249      1249  ...           1249   
          Benton Park               607       607  ...            607   
          Benton Park West          723       723  ...            723   
          Bevo Mill                1160      1160  ...           1160   
...                                 ...       ...  ...            ...   
ZEALAND   Midtown                     1         1  ...              1   
          The Great Ville             1         1  ...              1   
ZELLIE    Cardonlet                   3         3  ...              3   
alero     The Hill                    1         1  ...              1   
hilton    Downtown                    1         1  ...              1   

                            ILEADSStreet  LocationComment  LocationName  \
CADStreet NeighborhoodName                                                
          Academy                    867              135           206   
          Baden                     1249              142           189   
          Benton Park                607              106           100   
          Benton Park West           723              138            54   
          Bevo Mill                 1160              187           191   
...                                  ...              ...           ...   
ZEALAND   Midtown                      1                0             0   
          The Great Ville              1                0             0   
ZELLIE    Cardonlet                    3                1             0   
alero     The Hill                     1                0             0   
hilton    Downtown                     1                0             1   

                            Neighborhood  NeighborhoodPrimaryDistrict  \
CADStreet NeighborhoodName                                              
          Academy                    867                

In [14]:
crime_df

,CADAddress,CADStreet,CodedMonth,Count,Crime,ShortCrimeCode,UCRType,UCRCrime,DateOccured,Description,...,ILEADSStreet,LocationComment,LocationName,Neighborhood,NeighborhoodName,NeighborhoodPrimaryDistrict,NeighborhoodAddlDistrict,Latitude,Longitude,Year
0,4900,MAPLE,2008-04,1,177000,17,2,Sex Offenses,1/1/2000,SEX OFFNS-INCEST,...,MAPLE AVE,NaN,NaN,53.0,Fountain Park,8.0,NaN,38.658845,-90.260116,2008
2,4313,MARGARETTA,2008-04,1,115400,11,2,Fraud,1/1/2004,STLG BY DECEIT/IDENTITY THEFT REPORT,...,MARGARETTA AVE,NaN,NaN,68.0,O'Fallon,5.0,6.0,38.671754,-90.233544,2008
3,3939,OHIO,2008-04,1,115400,11,2,Fraud,1/1/2006,STLG BY DECEIT/IDENTITY THEFT REPORT,...,OHIO AVE,NaN,NaN,16.0,Dutchtown,1.0,NaN,38.583504,-90.230145,2008
4,2612,ANNIE MALONE DR,2008-04,1,91113,9,2,Other assaults,1/1/2007,SIMPLE ASSAULT-ADULT/NO INJURY,...,ANNIE MALONE DR,@ANNIE MALONE HOME,NaN,57.0,The Ville,8.0,NaN,38.659580,-90.236606,2008
5,5500,DELMAR,2008-04,1,21000,2,1,Forcible Rape,1/1/2008,RAPE -- FORCIBLE,...,DELMAR BLVD,NaN,NaN,49.0,Visitation Park,7.0,NaN,38.654057,-90.278155,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448995,4310,WILCOX,2015-09,1,65701,6,1,Larceny-theft,12/1/13 8:00,LARCENY-MTR VEH PARTS UNDER $500,...,WILCOX AVE,NaN,NaN,5.0,Bevo Mill,1.0,NaN,38.580835,-90.267600,2015
448996,3900,23RD,2015-09,1,21000,2,1,Forcible Rape,12/20/13 8:00,RAPE -- FORCIBLE,...,N 23RD ST,NaN,NaN,65.0,Hyde Park,5.0,NaN,35.801506,-93.265663,2015
448997,3320,LAWN,2015-09,1,91114,9,2,Other assaults,12/22/14 16:00,"ASSAULT, ADULT, AGE 17 AND UP-DOMESTIC",...,LAWN AVE,NaN,NaN,14.0,North Hampton,2.0,NaN,38.599216,-90.271246,2015
448998,,,2015-09,-1,31111,3,1,Robbery,12/26/14 21:20,ROBBERY-HIGHWAY /FIREARM USED/SUCCESSFUL,...,FLORISSANT,NaN,NaN,60.0,St. Louis Place,4.0,5.0,35.801506,-93.265663,2015


## Trends Per Year

## Offenses Per Neighborhood

## Heat Maps

In [15]:
# Import Api Key and other dependecies
from api_keys import g_key
import gmaps
import os

In [16]:
fifteen = pd.read_csv('Resources/2015.csv')  
fifteen = fifteen.loc[(pd.isna(fifteen['Latitude'])==False) 
                        & (pd.isna(fifteen['Longitude'])==False) 
                        & (pd.isna(fifteen['NeighborhoodName'])==False)]

fifteen['Year_calc'] = fifteen['CodedMonth'].str.split('-')

for index in fifteen.index:
    if pd.isna(fifteen['Year'][index]) == True:
        fifteen['Year'][index] =  fifteen['Year_calc'][index][0]
        
fifteen.drop('Year_calc', inplace=True, axis=1)

one_five = fifteen[["UCRType","UCRCrime","Count","Latitude","Longitude"]]


FileNotFoundError: [Errno 2] No such file or directory: 'Data/2015.csv'

In [ ]:
#Create data sets for Larceny type of crime 
larce = one_five.loc[one_five['UCRCrime'] == 'Larceny-theft']
larce = larce.loc[larce["Count"] >= 1]
#Create locations
lat_lng_lar = larce[["Latitude","Longitude"]]
#Create Weight
weight = larce["Count"]

#Create data sets for Aggravated Assault type of crime 
a_a = one_five.loc[one_five['UCRCrime'] == 'Aggravated Assault']
a_a = a_a.loc[a_a["Count"] >= 1]
#Create locations
lat_lng_a = a_a[["Latitude","Longitude"]]
#Create Weight
weight_a = a_a["Count"]



In [ ]:
#Use fig and layer to map on data set 
fig_one = gmaps.figure()

heat_layer_one = gmaps.heatmap_layer(lat_lng_lar, weights=weight, point_radius = 5)

fig_one.add_layer(heat_layer_one)

#Larceny in STL in year 2015
fig_one

In [ ]:
fig_two = gmaps.figure()

heat_layer_two = gmaps.heatmap_layer(lat_lng_a, weights=weight_a, point_radius = 5)

fig_two.add_layer(heat_layer_two)

fig_two